In [ ]:
#import packages
import pandas as pd
import datetime as dt
import numpy as np
import unidecode 
from unidecode import unidecode

import os

In [ ]:
# dtype_mapping = {'Date de création': 'datetime64[D]'}

# raw_list = pd.read_excel('C:/Users/BGE/Google Drive/GIT/listeCreations 2021 to 2024 02 29.xlsx', dtype=dtype_mapping)
raw_list = pd.read_excel('C:/Users/BGE/Google Drive/GIT/listeCreations 2021 to 2024 02 29.xlsx')
raw_list.info()
raw_list.head(50)
raw_list['Date de création'].dtype
# raw_list['Date de création'].astype('<M8[D]')
# raw_list['Date de création'] = pd.to_datetime(raw_list['Date de création'])
# raw_list['Date de création']= raw_list['Date de création'].dt.floor('D')
raw_list['Date de création'] = pd.to_datetime(raw_list['Date de création'], unit='D', origin='1899-12-30')


In [ ]:
raw_list

In [ ]:
#replace "nan" by "unknown"
replace_nan = lambda x: x.replace(np.nan, 'unknown')

columns_to_check = [ column_str for column_str in raw_list.columns if raw_list.columns.dtype == 'object' ]
mask = raw_list.columns.isin(columns_to_check)

# Filter DataFrame colums using the mask
columns_to_check = raw_list.columns[mask]

# Apply the replace_nan function only to columns that are not of type 'object'
raw_list[columns_to_check] = raw_list[columns_to_check].apply(replace_nan)

raw_list


In [ ]:
#concatenate "Nom" and "Prenom"
raw_list['Entrepreneur'] = raw_list['Prénom'] + ' ' + raw_list['Nom']
raw_list.drop(columns=['Prénom', 'Nom'], inplace=True)

#reordered columns
raw_list_col_reordered = ['Lieu accueil', 'Entrepreneur', 'Lieu de suivi', 'Raison sociale', 'Date de création',
       'Type de création', 'Statut fiscal', 'Banque', 'Expert comptable',
       'Phase STARTER Post Création acceptée', 'total anté (h)',
       'total anté Starter (h)', 'ref. Entreprise', 'total h suivi', 'CAPE',
       'Durée (h) consulting', 'Durée (h) formation créa', 'partenaire'
       ]
raw_list = raw_list[raw_list_col_reordered]
raw_list

In [ ]:
# harmonize between tables coaches names

#split ref Entreprise column, join in inverse way firstname / name
raw_list[['Name ref', 'Firstname ref']] = raw_list['ref. Entreprise'].str.rsplit(' ', n=1, expand=True)
raw_list['Conseiller'] = raw_list['Firstname ref'] + ' ' + raw_list['Name ref']

#erase old ref Entreprise column and splitted columns
raw_list.drop(columns=['ref. Entreprise', 'Name ref', 'Firstname ref'])

#reorder columns
raw_list_columns_reordered = ['Lieu accueil', 'Entrepreneur', 'Lieu de suivi', 'Conseiller', 'Raison sociale',
       'Date de création', 'Type de création', 'Statut fiscal', 'Banque',
       'Expert comptable', 'Phase STARTER Post Création acceptée',
       'total anté (h)', 'total anté Starter (h)',
       'total h suivi', 'CAPE', 'Durée (h) consulting',
       'Durée (h) formation créa', 'partenaire']
cleaned_list = raw_list[raw_list_columns_reordered]
cleaned_list

In [ ]:
# Define lambdas for transformations
lower_index = lambda x: x.lower() if isinstance(x, str) else x
erase_indent = lambda x: x.replace('-', '') if isinstance(x, str) else x
strip_index = lambda x: x.strip() if isinstance(x, str) else x
accents_strip = lambda x: unidecode(x) if isinstance(x, str) else x  # Apply unidecode only to string values

# Define function to decode string using multiple encodings
def decode_string(original_string):
    if isinstance(original_string, int):  # Check if input is an integer (likely an Excel date)
        return original_string  # Return the integer as is
    elif isinstance(original_string, float):  # Check if input is a float
        return original_string  # Return the float as is
    else:
        encodings_to_try = ['utf-8', 'latin-1', 'windows-1252']
        encoded_string = original_string.encode('utf-8')  # Encode once before the loop
        for encoding in encodings_to_try:
            try:
                decoded_text = encoded_string.decode(encoding)
                return decoded_text
            except UnicodeDecodeError:
                print(f"Failed to decode with {encoding}")
        return original_string  # Return original value if all decoding attempts fail

# Identify columns with dtype 'object' (string columns)
str_columns_list = [str_column for str_column in raw_list.columns if raw_list[str_column].dtype == 'object']

# Apply transformations to selected string columns
for column in str_columns_list:
    cleaned_list[column] = raw_list[column].apply(decode_string).apply(lower_index).apply(erase_indent).apply(strip_index).apply(accents_strip)

# Output the transformed DataFrame
cleaned_list



In [ ]:
#csv export 
cleaned_list.to_csv('cleaned_list_biz_started_from2021_until29022024.csv', index = False)